#### torchtext 라이브러리로 텍스트 분류 <hr>

-   [1]단계 - 데이터 전처리 : 숫자형식으로 변환하는 것 까지
-   [2]단계 - 모델 구현


-   [1-1] 데이터 준비 => 내장 데이터셋 활용
    -   AG_NEWS 데이터셋 반복자 : 레이블(label) + 문장의 튜플(tuple) 형태


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


from torchtext.datasets import AG_NEWS


### ===> DataPipe 타입 >>> iterator 타입 형변환


train_iter = iter(AG_NEWS(split="train"))


In [2]:
# 데이터 확인 => (label, text), label 1~4
next(train_iter)


(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

-   [2] 데이터 처리 파이프라인 준비 <hr>
    -   어휘집(vocab), 단어 벡터(word vector), 토크나이저(tokenizer)
    -   가공되지 않은 텍스트 문자열에 대한 데이터 처리 빌딩 블록
    -   일반적인 NLP 데이터 처리
        -   첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성
            -   ==> 토큰 목록 또는 반복자 받는 내장 팩토리 함수(factory function) : `build_vocab_from_iterator`
        -   사용자는 어휘집에 추가할 특수 기호(special symbol) 전달 가능


In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

### ==> 토크나이저 생성
tokernizer = get_tokenizer("basic_english")

### ===> 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split="train")


In [4]:
### ===> 토큰 제네레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트 --> 텍스트 토큰화
        yield tokernizer(text)


In [5]:
### ===> 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

### ===> <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab["<unk>"])


In [6]:
vocab(["<unk>", "here", "is", "an", "example"])


[0, 475, 21, 30, 5297]

In [7]:
### ===> 텍스트 >>>> 정수 인코딩
text_pipeline = lambda x: vocab(tokernizer(x))

### ===> 레이블 >>> 정수 인코딩 (0~3)
label_pipeline = lambda x: int(x) - 1


-   [3] 데이터 배치(batch)와 반복자 생성 <hr>
    -   torch.utils.data.DataLoader : getitem(), len() 구현한 맵 형태(map-style)
    -   collate_fn() : DataLoader로부터 생성된 샘플 배치 함수
        -   입력 : DataLoader에 배치 크기(batch_size)가 있는 배치(batch) 데이터


In [8]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### ===> 배치크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    # 배치크기만큼의 라벨, 텍스트, 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0]

    # 1개씩 뉴스기사, 라벨 추출해서 저장
    for _label, _text in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))

    # 텐서화 진행
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)


In [9]:
train_iter = AG_NEWS(split="train")
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)


In [10]:
### ===> 분류 클래스 수와 단어사전 개수
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f"num_class : {num_class}    vocab_size : {vocab_size}")


num_class : 4    vocab_size : 95811


In [11]:
for label, text, offset in dataloader:
    print(
        label,
        end="\n=============================================================================\n",
    )
    print(
        text,
        end="\n=============================================================================\n",
    )
    print(offset)
    break


tensor([2, 2, 2, 2, 2, 2, 2, 2])
tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27,    14,    27,   

In [12]:
### ==> 모델 설계
### 입력층 : EmbeddingBag Layer - 레이어와 분류(classification) 목정을 위한 선형 레이어, 텍스트의 길이는 오프셋(offset)
### 은닉층 : Linear - 4개 클래스 분류
class TextModel(nn.Module):
    def __init__(self, VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS) -> None:
        super().__init__()
        # 모델 구성 층 정의
        self.embedding = nn.EmbeddingBag(VOCAB_SIZE, EMBEDD_DIM, sparse=False)
        self.rnn = nn.RNN(EMBEDD_DIM, HIDDEN_SIZE, batch_first=True)
        self.fc = nn.Linear(HIDDEN_SIZE, NUM_CLASS)
        self.init_weights()

    # 가중치 초기화
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    # 순방향 학습 진행
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        output, _ = self.rnn(embedded)
        return self.fc(output)


In [13]:
train_iter = AG_NEWS(split="train")
### ==> 학습 관련 하이퍼파라미터와 인스턴스
HIDDEN_SIZE = 3
EMBEDD_DIM = 64
VOCAB_SIZE = len(vocab)
NUM_CLASS = len(set([label for label, _ in train_iter]))
EPOCHS = 10
LR = 5
BATCH_SIZE = 64


In [14]:
### ==> 학습 관련 인스턴스
MODEL = TextModel(VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS).to(device)

CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = optim.SGD(params=MODEL.parameters(), lr=LR)
SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER, 1.0, 0.1)


-   [5] 학습 관련 함수들 <hr>


In [15]:
def train(model, deataloader, optimizer, criterion, epoch):
    # 학습 모드
    model.train()

    # 학습 평가 관련 변수들
    total_acc, total_count = 0, 0
    log_interval = 300

    # 배치 학습 진행
    for idx, (label, text, offsets) in enumerate(dataloader):
        label, text, offsets = label.to(device), text.to(device), offsets.to(device)

        # 학습진행
        predicted_label = model(text, offsets)
        # print(f'predicted_label : {predicted_label.shape}, label : {label.shape}')

        # 손실 계산 및 W,b 업데이트
        optimizer.zero_grad()
        loss = criterion(predicted_label, label)
        loss.backward()
        # 기울기 소실 및 폭주 예방을 위한 양극단 값 자르기
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        # 배치 학습 평가
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            print(f"epoch : {epoch} batch : {idx} loss : {loss.item()}")
            print(f"Accuracy : {total_acc/total_count}")
            total_acc, total_count = 0, 0


In [16]:
### ===> 검증 및 테스트 함수
def evaluate(model, dataloader, criterion):
    model.eval()

    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

    return total_acc / total_count


In [17]:
### ====> 예측 함수
def predict(model, text, text_pipeline):
    with torch.no_grad():
        # 토큰화 > 정수 변환 > 텐서
        text = torch.tensor(text_pipeline(text), dtype=torch.int64).to(device)
        text = text.unsqueeze(0)
        offsets = torch.tensor([0]).to(device)
        predicted_label = model(text, offsets)
        return predicted_label.argmax(1).item() + 1


In [18]:
train_iter = AG_NEWS(split="train")
test_iter = AG_NEWS(split="test")
trainDL = DataLoader(train_iter, batch_size=64, shuffle=False, collate_fn=collate_batch)
testDL = DataLoader(train_iter, batch_size=64, shuffle=False, collate_fn=collate_batch)


In [19]:
# 학습 진행
for epoch in range(1, EPOCHS + 1):
    train(MODEL, trainDL, OPTIMIZER, CRITERION, epoch)
    accu_val = evaluate(MODEL, testDL, CRITERION)
    print(f"epoch : {epoch} Accuracy : {accu_val}")
    SCHEDULER.step()


c:\Users\KDP-25\.conda\envs\Torch_NLP38\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


epoch : 1 batch : 300 loss : 1.255967140197754
Accuracy : 0.39908637873754155
epoch : 1 batch : 600 loss : 0.37646058201789856
Accuracy : 0.42083333333333334
epoch : 1 batch : 900 loss : 1.9208753108978271
Accuracy : 0.5083333333333333
epoch : 1 batch : 1200 loss : 2.494374990463257
Accuracy : 0.5875
epoch : 1 batch : 1500 loss : 0.668969988822937
Accuracy : 0.6508333333333334
epoch : 1 batch : 1800 loss : 2.5411782264709473
Accuracy : 0.7129166666666666
epoch : 1 batch : 2100 loss : 0.26841455698013306
Accuracy : 0.7266666666666667
epoch : 1 batch : 2400 loss : 0.5886327028274536
Accuracy : 0.7558333333333334
epoch : 1 batch : 2700 loss : 0.6617964506149292
Accuracy : 0.7670833333333333
epoch : 1 batch : 3000 loss : 2.2276809215545654
Accuracy : 0.7795833333333333
epoch : 1 batch : 3300 loss : 0.46741750836372375
Accuracy : 0.80125
epoch : 1 batch : 3600 loss : 0.5135096311569214
Accuracy : 0.8075
epoch : 1 batch : 3900 loss : 0.8300088047981262
Accuracy : 0.8091666666666667
epoch : 1